In [2]:
import pandas as pd

from CitNet import DisambName as DN

import time
import os

from jyquickhelper import add_notebook_menu

In [33]:
add_notebook_menu()

# Disambiguation

In [4]:
# Set path
path=os.path.join(os.getcwd(), "Tables")

In [9]:
# Load data
attrs = pd.read_csv(path + "/attrs.csv")
attrs.head()

,url,title,authors,date,jel_code,keywords,editor,journal,article_id
0,https://ideas.repec.org/a/oup/qjecon/v1y1886i1...,The Reaction in Political Economy,Charles F. Dunbar,1886-02-02,NaN,NaN,oup,qjecon,v1y1886i1p1-27..html
1,https://ideas.repec.org/a/oup/qjecon/v1y1886i1...,Private Monopolies and Public Rights,Arthur T. Hadley,1886-02-02,NaN,NaN,oup,qjecon,v1y1886i1p28-44..html
2,https://ideas.repec.org/a/oup/qjecon/v1y1886i1...,Silver Before Congress in 1886,S. Dana Horton,1886-02-02,NaN,NaN,oup,qjecon,v1y1886i1p45-75..html
3,https://ideas.repec.org/a/oup/qjecon/v1y1886i1...,"The Arithmetic, Geometric, and Harmonic Means",F. Coggeshall,1886-02-02,NaN,NaN,oup,qjecon,v1y1886i1p83-86..html
4,https://ideas.repec.org/a/oup/qjecon/v1y1886i1...,Legislation for Labor Arbitration,H. M. Williams,1886-02-02,NaN,NaN,oup,qjecon,v1y1886i1p86-91..html


## Disambig Authors dataset

In [10]:
# Parse authors from string to list (inplace)
attrs["authors_list"] = attrs["authors"].apply(DN.authors_parser)

In [41]:
# Stack all authors in a list (w/o duplicates)
authors = []
for author in attrs["authors_list"]:
    authors += author
authors = list(set(authors))

In [42]:
# Create a dataframe for authors
df_authors = pd.DataFrame(authors, columns=["original"])
# Create a column with authors in a uniform format
df_authors["uniformat"] = df_authors["original"].apply(DN.uniformize_names)

In [26]:
%%time
# Map authors
#cleaned = DN.map_authors(df_authors_reduced, 0.12)
cleaned = DN.map_authors(df_authors, 0.12)

/Users/cyrilverluise/Documents/DS/S2/NetworkA/EconomicsPapersNetwork/CitNet/DisambName.py:139: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  "original",
/Users/cyrilverluise/Documents/DS/S2/NetworkA/EconomicsPapersNetwork/CitNet/DisambName.py:140: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  "uniformat",
/Users/cyrilverluise/Documents/DS/S2/NetworkA/EconomicsPapersNetwork/CitNet/DisambName.py:141: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  "equivalent"],


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
CPU times: user 10min 20s, sys: 16.9 s, total: 10min 37s
Wall time: 10min 42s


In [27]:
# Check the output (cases where multiple )
print("There were {0} names, only {1} are left.".format(len(df_authors_reduced), len(cleaned)))
cleaned[cleaned.equivalent.apply(lambda x: len(x)) > 1].head()

There were 2000 names, only 39636 are left.


,original,uniformat,equivalent
0,Maximo Torero,Maximo Torero,"[Maximo Torero, Torero, Maximo]"
5,Sheri Tice,Sheri Tice,"[Sheri Tice, Tice, Sheri]"
7,Taylor Nadauld,Taylor Nadauld,"[Taylor Nadauld, Nadauld, Taylor D.]"
10,Lawrence M. Kahn,Lawrence M. Kahn,"[Lawrence M. Kahn, Kahn, Lawrence M, Lawrence ..."
11,Liam P. Ebrill,Liam P. Ebrill,"[Liam P. Ebrill, Ebrill, Liam P.]"


In [28]:
# Sort the resulting modified authors dataframe by "uniformat" and reset its index
cleaned = cleaned.sort_values(by="uniformat")
cleaned = cleaned.reset_index(drop=True)

In [31]:
# Save output
cleaned.to_csv(path + "/authors.csv", index=False)

In [34]:
#del cleaned

## Attrs Disambig Name asssignment 

In [5]:
# Reload authors (copy)
cleaned_cop = pd.read_csv(path + "/authors.csv", encoding = "ISO-8859-1")

In [6]:
# Preprocess equivalent
cleaned_cop["equivalent"] = cleaned_cop["equivalent"].apply(DN.str_to_list)
#cleaned_bis = unpack_authors_list(cleaned_cop) #???

In [7]:
# Find authors indexes for each paper in attrs (WARNING : TAKES A LITTLE MORE THAN AN HOUR)
maxs_eq = max(cleaned_cop.equivalent.apply(lambda x: len(x)))
eqs_cols = ["equivalent_" + str(i) for i in range(0, maxs_eq)]

In [15]:
%time
attrs["authors_nos"] = attrs["authors_list"].apply(lambda x: DN.author_corresp(cleaned_cop, eqs_cols, x))
# ERROR

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 14.8 µs


KeyError: 'equivalent_0'

In [ ]:
attrs.to_csv(path + "attrs_nos_sec.csv")